In [1]:
import keras
import numpy as np
from matplotlib import pyplot as plt
import math
import pandas
from keras import Sequential
from keras.layers import *
from keras.models import Model

Using TensorFlow backend.


input1 = Missing image (Masked by cloud)

input2 = Referenced images (cloud-free)

In [2]:
input1 = Input(shape=(400,400,1), name='input1')
input2 = Input(shape=(400,400,1), name='input2')

In [3]:
conv_1 = Conv2D(30, (3, 3),padding='same')(input1)
conv_2 = Conv2D(30, (3, 3),padding='same')(input2)
conv_1

<tf.Tensor 'conv2d_1/BiasAdd:0' shape=(?, 400, 400, 30) dtype=float32>

In [4]:
concat_1_2 = concatenate([conv_1, conv_2], axis=-1)
concat_1_2 = Activation('relu')(concat_1_2)
concat_1_2

<tf.Tensor 'activation_1/Relu:0' shape=(?, 400, 400, 60) dtype=float32>

By default, stride = 1, padding=1. Trying to not manually implement?

In [5]:
feature_3 = Conv2D(filters=20, kernel_size=(3, 3),padding='same')(concat_1_2)
feature_5 = Conv2D(filters=20, kernel_size=(5, 5),padding='same')(concat_1_2)
feature_7 = Conv2D(filters=20, kernel_size=(7, 7),padding='same')(concat_1_2)
feature_5

<tf.Tensor 'conv2d_4/BiasAdd:0' shape=(?, 400, 400, 20) dtype=float32>

In [6]:
concat_3_5_7 = concatenate([feature_3, feature_5, feature_7])
concat_3_5_7 = Activation('relu')(concat_3_5_7)
concat_3_5_7

<tf.Tensor 'activation_2/Relu:0' shape=(?, 400, 400, 60) dtype=float32>

In [7]:
sum0 = add([concat_1_2, concat_3_5_7])
sum0

<tf.Tensor 'add_1/add:0' shape=(?, 400, 400, 60) dtype=float32>

In [8]:
conv1 = Conv2D(filters=60, kernel_size=(3,3), padding='same',activation='relu')(sum0)
conv2 = Conv2D(filters=30, kernel_size=(3,3), padding='same',activation='relu')(conv1)
conv2

<tf.Tensor 'conv2d_7/Relu:0' shape=(?, 400, 400, 30) dtype=float32>

In [9]:
sum1 = add([conv_2, conv2])
sum1

<tf.Tensor 'add_2/add:0' shape=(?, 400, 400, 30) dtype=float32>

In [10]:
conv3 = Conv2D(filters=60, kernel_size=(3,3), dilation_rate=2, padding='same',activation='relu')(sum1)
conv3

<tf.Tensor 'conv2d_8/Relu:0' shape=(?, 400, 400, 60) dtype=float32>

In [11]:
conv4 = Conv2D(filters=60, kernel_size=(3,3), dilation_rate=3, padding='same',activation='relu')(conv3)
conv4

<tf.Tensor 'conv2d_9/Relu:0' shape=(?, 400, 400, 60) dtype=float32>

In [12]:
conv5 = Conv2D(filters=60, kernel_size=(3,3), dilation_rate=2, padding='same',activation='relu')(conv4)
conv5

<tf.Tensor 'conv2d_10/Relu:0' shape=(?, 400, 400, 60) dtype=float32>

In [13]:
sum2 = add([conv3, conv5])
sum2

<tf.Tensor 'add_3/add:0' shape=(?, 400, 400, 60) dtype=float32>

In [14]:
conv6 = Conv2D(filters=60, kernel_size=(3,3), padding='same',activation='relu')(sum2)
conv6

<tf.Tensor 'conv2d_11/Relu:0' shape=(?, 400, 400, 60) dtype=float32>

In [15]:
conv7 = Conv2D(filters=1, kernel_size=(3,3), padding='same')(conv6)
conv7

<tf.Tensor 'conv2d_12/BiasAdd:0' shape=(?, 400, 400, 1) dtype=float32>

In [16]:
model = Model([input1, input2], conv7)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input2 (InputLayer)             (None, 400, 400, 1)  0                                            
__________________________________________________________________________________________________
input1 (InputLayer)             (None, 400, 400, 1)  0                                            
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 400, 400, 30) 300         input2[0][0]                     
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 400, 400, 30) 300         input1[0][0]                     
__________________________________________________________________________________________________
concatenat